In [1]:
import os #for handling file and directories
from PIL import Image #image preprocesing and 
import numpy as np# store data in array
import torch#Core library for training and creating model
from torch.utils.data import Dataset, DataLoader#for loading Datasets and managing batches
import torch.nn as nn#for complex neural network architecture
import torch.optim as optim # for optimizing model's parameter
from sklearn.model_selection import train_test_split# for spliting the dataset into training and testing data
import pandas as pd
import albumentations as A
import io
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


c:\Users\shash\OneDrive\Desktop\practicing_DS\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classes = {
  0:"Cat",
  1:'Dawg'
}

In [3]:
splits = {'train': 'Data/trainDataSet.parquet', 'test': 'Data/testDataSet.parquet'}
trainingDataSet = pd.read_parquet(splits["train"])
testingDataSet = pd.read_parquet(splits['test'])

In [17]:
testingDataSet["image"][0]['bytes']

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\n\x07\x07\x08\x07\x06\n\x08\x08\x08\x0b\n\n\x0b\x0e\x18\x10\x0e\r\r\x0e\x1d\x15\x16\x11\x18#\x1f%$"\x1f"!&+7/&)4)!"0A149;>>>%.DIC<H7=>;\xff\xdb\x00C\x01\n\x0b\x0b\x0e\r\x0e\x1c\x10\x10\x1c;("(;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\xff\xc0\x00\x11\x08\x01\x9f\x01\xf2\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe

In [5]:
""" print(type(trainingDataSet)) """
""" print(trainingDataSet.columns) """
print(testingDataSet.columns)

Index(['image', 'labels'], dtype='object')


In [6]:
class imageDataSet(Dataset) :
  def __init__(self,trainingDataSet,transform = None,target_transform = None):
    super().__init__()
    self.trainingDataSet_path = trainingDataSet["image"]
    self.trainingDataSet_label = trainingDataSet["labels"]
    self.transform = transform
    self.target_transform = target_transform
    
  def __len__(self):
    return len(self.trainingDataSet_path)
  
  def __getitem__(self, index):
    
    img_path = self.trainingDataSet_path.iloc[index]
    label = self.trainingDataSet_label.iloc[index]
    image_stream = io.BytesIO(img_path['bytes'])
    
    image = Image.open(image_stream).convert('RGB')
    
    if self.transform:
      image = self.transform(image)
      
    if self.target_transform:
      label = self.target_transform(label)
    
    return  image, label

In [7]:
#Transformation (recommended)
from torchvision import transforms

compose = transforms.Compose([
  transforms.Resize((64,64)),
  transforms.ToTensor(),
])

In [8]:
# Test my imageDataSet class for errors
train_DataSet = imageDataSet(trainingDataSet=trainingDataSet,transform=compose)
image, label = next(iter(train_DataSet))
train_Loader = DataLoader(train_DataSet,batch_size = 5,shuffle=True)

test_DataSet = imageDataSet(trainingDataSet=testingDataSet,transform=compose)
image, label = next(iter(test_DataSet))
test_Loader = DataLoader(test_DataSet,batch_size=1,shuffle=True)


In [9]:
import torch.nn.functional as F

class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=1)
    self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
    self.conv2 = nn.Conv2d(16,32,3,1,1)
    self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
    self.flatten = nn.Flatten(start_dim=1, end_dim=-1)
    self.fc1 = nn.Linear(32*16*16,128)
    self.relu = nn.ReLU(True)
    self.fc2 = nn.Linear(128,1)
    
  def forward(self,x):
   
    x = self.conv1(x) 
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x
      
model = Network()   
    

In [10]:
#Loss Function using crossEntropyLoss
loss = nn.BCEWithLogitsLoss()

In [11]:
#Optimizer using Adams
optimizer = optim.Adam(model.parameters(),lr=(1e-4))

In [12]:
#Evaluate Model
from tqdm import tqdm
@torch.no_grad()
def evaluate_model(test):
	model.eval()
	print("Running Validation")
	correct = 0
	for x,y in tqdm(test):
		x = x.to(device)
		y = y.view(-1, 1).float().to(device)
		out = torch.sigmoid(model(x))
		prediction = (out >= 0.5).int()
		correct += prediction.item() == y.item()
		accuracy = (correct/len(test))*100
	print(f"Accuracy : {accuracy}")
	return accuracy

In [13]:
#Train Model

max_accuracy = 0
EPOCHS = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for epoch in range(EPOCHS):
  sum_loss = 0
  model.train()
  for image,label in tqdm(train_Loader):
    image = image.to(device)
    label = label.to(device).float()
    
    output = model(image)
    output = output.squeeze()
    Loss = loss(output,label)
    
    optimizer.zero_grad()
    Loss.backward()
    optimizer.step()
    
    sum_loss += Loss.item()
  
    
  print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {sum_loss/len(train_Loader)}")
  acc = evaluate_model(test_Loader)
  print(f"Accuracy:{acc}%")

100%|██████████| 1600/1600 [00:23<00:00, 67.97it/s]


Epoch [1/50], Loss: 0.653089577499777
Running Validation


100%|██████████| 2000/2000 [00:03<00:00, 545.09it/s]


Accuracy : 68.65
Accuracy:68.65%


100%|██████████| 1600/1600 [00:23<00:00, 66.80it/s]


...

Accuracy : 73.8
Accuracy:73.8%


100%|██████████| 1600/1600 [00:22<00:00, 70.00it/s]


Epoch [50/50], Loss: 0.00012453207999554294
Running Validation


100%|██████████| 2000/2000 [00:03<00:00, 593.18it/s]

Accuracy : 73.55000000000001
Accuracy:73.55000000000001%


In [18]:
torch.save(model.state_dict(),"BinaryClassifier.pth")

In [20]:
image = Image.open("./dog.webp")
image_Range = np.array(image)
print("MIN",image_Range.min())
print("MAX",image_Range.max())

MIN 0
MAX 255


In [28]:
image_paths = "./dog.webp"
image = Image.open(image_paths).convert("RGB")
pixel_sum = np.zeros(3)
pixel_squared_sum = np.zeros(3)
num_pixel = 0

for img_path in image_paths:
  
  image_array = np.array(image) / 255.0  # Scale to [0, 1]
  pixel_sum += image_array.mean(axis=(0, 1))  # Sum over H, W for R, G, B
  pixel_squared_sum += (image_array ** 2).mean(axis=(0, 1))  # Squared sum
  num_pixel += 1

mean = pixel_sum / num_pixel
std = (pixel_squared_sum / num_pixel - mean ** 2) ** 0.5  # Standard deviation

print("Mean:", mean)
print("Std:", std)


Mean: [0.75128635 0.73187373 0.66569826]
Std: [0.16277556 0.16570601 0.2318707 ]
